Implementation of a __LangGraph__ workflow with these nodes:

- InputNode $\rightarrow$ receives player input.
- CheckContextNode $\rightarrow$ gets context from MCP.
- ToolDecisionNode $\rightarrow$ checks if input contains factual query (e.g., keywords: 'what is', 'who is', 'types of', etc.)
- ToolCallNode $\rightarrow$ performs TOOL_CALL via MCP if needed.
- PromptAssemblyNode $\rightarrow$ assembles prompt (Adam's persona + context + player message).
- LLMNode $\rightarrow$ gets GPT response.
- OutputNode $\rightarrow$ logs final message and updates MCP with new dialogue turn.

__using LangGraph and OpenAI__ 

### Code workflow
This code sets up a FastAPI server that:
- Stores a conversation with the player and Adam.
- Manages memory so it fits within a token limit.
- Can summarize old conversations.
- Can reset the conversation.
- Can make tool calls (Wikipedia search) if needed.

This code is LangGraph's dialogue engine that powers the NPC "Adam"!
1. It simulates a conversation loop where:
2. It takes player input.
3. It fetches past context from the FastAPI server.
4. It decides if a Wikipedia tool call is needed.
5. It assembles a prompt including persona + context + tool results (if any).
6. It sends the final prompt to GPT (using ChatOpenAI) to generate Adam's reply.
7. It prints _Adam's_ reply.


## Breaking It Down

| Component | What it does |
|:---|:---|
| `persona` | A fixed description of Adam's character. |
| `ChatOpenAI` | Calls GPT-3.5-Turbo to generate Adam's responses. |
| `StateGraph` | A directed graph where each node is a step in the conversation logic. |
| `Nodes` | Functions like `input_node`, `check_context_node`, etc. Each represents a step. |
| `ToolNode` (optional) | (Unused here) - could have been used if you had tools as objects. |
| `requests` | Used to communicate with your FastAPI memory and tool call server (MCP). |


In [ ]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph import ToolNode
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
import requests

In [ ]:
# Define Adam's persona
persona = "Adam is a wise, centuries-old sage from the northern isles. He speaks with empathy and shares ancient lore."

In [ ]:
# Initialize the LLM (GPT-3.5-Turbo)
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Node Definitions

In [ ]:
# 1. Input Node: Get player input
def input_node(state):
    state["user_input"] = input("You: ")  # Get user input
    return state, "check_context"         # Move to check_context node

# 2. Context Fetch Node: Fetch conversation memory
def check_context_node(state):
    res = requests.get("http://localhost:8000/get_context")  # Call MCP server
    state["context"] = res.json()["context"]                 # Save conversation context
    return state, "tool_decision"                            # Move to tool_decision node

# 3. Tool Decision Node: Should we call Wikipedia?
def tool_decision_node(state):
    question = state["user_input"].lower()
    # Check if the input contains factual keywords
    if any(kw in question for kw in ["what is", "who is", "types of", "genres", "explain"]):
        return state, "tool_call"  # If factual, call tool
    return state, "assemble_prompt"  # Otherwise, move to assemble_prompt

# 4. Tool Call Node: Call Wikipedia if needed
def tool_call_node(state):
    query = state["user_input"]
    res = requests.get(f"http://localhost:8000/tool_call", params={"query": query})  # Wikipedia API through MCP
    state["tool_result"] = res.json()["result"]  # Save Wikipedia result
    return state, "assemble_prompt"

# 5. Assemble Prompt Node: Build final prompt for GPT
def assemble_prompt_node(state):
    template = PromptTemplate.from_template(
        "{persona}\n\nConversation so far:\n{history}\n\nPlayer: {user_input}\nAdam:"
    )

    # Format conversation history into a string
    history = "\n".join([f'{m["role"]}: {m["content"]}' for m in state["context"]])

    # If we have a tool result, add it to history
    if "tool_result" in state:
        history += f"\nTool: {state['tool_result']}"

    # Fill the prompt template
    prompt = template.format(persona=persona, history=history, user_input=state["user_input"])
    
    state["final_prompt"] = prompt
    return state, "llm"

# 6. LLM Node: Generate Adam's reply
def llm_node(state):
    response = llm.invoke(state["final_prompt"])  # Call GPT
    state["llm_output"] = response.content        # Save GPT's output
    return state, "output"

# 7. Output Node: Print Adam's reply and save to memory
def output_node(state):
    print(f"Adam: {state['llm_output']}")  # Show GPT's reply

    # Save the conversation back into memory via MCP
    requests.post("http://localhost:8000/add_message", json={"role": "user", "content": state["user_input"]})
    requests.post("http://localhost:8000/add_message", json={"role": "assistant", "content": state["llm_output"]})
    
    return state, END  # End the workflow

# Build the Graph

In [ ]:
workflow = StateGraph()

# Add all nodes to the graph
workflow.add_node("input", input_node)
workflow.add_node("check_context", check_context_node)
workflow.add_node("tool_decision", tool_decision_node)
workflow.add_node("tool_call", tool_call_node)
workflow.add_node("assemble_prompt", assemble_prompt_node)
workflow.add_node("llm", llm_node)
workflow.add_node("output", output_node)

# Set the starting point
workflow.set_entry_point("input")

# Define edges (state transitions)
workflow.add_edge("input", "check_context")
workflow.add_edge("check_context", "tool_decision")
workflow.add_edge("tool_decision", "tool_call")
workflow.add_edge("tool_decision", "assemble_prompt")
workflow.add_edge("tool_call", "assemble_prompt")
workflow.add_edge("assemble_prompt", "llm")
workflow.add_edge("llm", "output")

# Compile the graph into an executable application
app = workflow.compile()

# Run the Dialogue Loop

In [ ]:
if __name__ == "__main__":
    while True:
        app.invoke({})  # Start the conversation loop
